In [1]:
import regex
import numpy as np
import pandas as pd

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
from wordcloud import WordCloud

C:\Users\Chuan Wu\anaconda3\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [2]:
df=pd.read_csv("dataset/Text.csv",encoding='latin1')

In [3]:
df.shape

(19883, 7)

In [4]:
df.head()

,Unnamed: 0,post_text,followers,friends,favourites,statuses,label
0,0,It's just over 2 years since I was diagnosed w...,84,211,251,837,1
1,1,"It's Sunday, I need a break, so I'm planning t...",84,211,251,837,1
2,2,Awake but tired. I need to sleep but my brain ...,84,211,251,837,1
3,3,RT @SewHQ: #Retro bears make perfect gifts and...,84,211,251,837,1
4,4,Itâs hard to say whether packing lists are m...,84,211,251,837,1


In [5]:
data = df[df['label']==1]
data

,Unnamed: 0,post_text,followers,friends,favourites,statuses,label
0,0,It's just over 2 years since I was diagnosed w...,84,211,251,837,1
1,1,"It's Sunday, I need a break, so I'm planning t...",84,211,251,837,1
2,2,Awake but tired. I need to sleep but my brain ...,84,211,251,837,1
3,3,RT @SewHQ: #Retro bears make perfect gifts and...,84,211,251,837,1
4,4,Itâs hard to say whether packing lists are m...,84,211,251,837,1
...,...,...,...,...,...,...,...
9939,9995,"RT @CSLewisU: ""We all agreed that forgiveness ...",78,175,664,1148,1
9940,9996,Can't believe it's been a year. #blastyblast #...,78,175,664,1148,1
9941,9997,"One year ago, a journey began. I never knew th...",78,175,664,1148,1
9942,9998,I found some new hammocking friends in the par...,78,175,664,1148,1


In [6]:
import nltk;
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data_words = list(sent_to_words(data['post_text']))

print(data_words[0])

['it', 'just', 'over', 'years', 'since', 'was', 'diagnosed', 'with', 'anxiety', 'and', 'depression', 'today', 'taking', 'moment', 'to', 'reflect', 'on', 'how', 'far', 've', 'come', 'since']


In [8]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

print(bigram_mod[data_words[0]])

['it', 'just', 'over', 'years', 'since', 'was_diagnosed', 'with', 'anxiety', 'and', 'depression', 'today', 'taking', 'moment', 'to', 'reflect', 'on', 'how', 'far', 've', 'come', 'since']


In [9]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
data_words_nostops = remove_stopwords(data_words)

data_words_bigrams = make_bigrams(data_words_nostops)

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

C:\Users\Chuan Wu\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


['year', 'diagnose', 'anxiety', 'depression', 'today', 'take', 'moment', 'reflect', 'far', 'come', 'since']


In [11]:
id2word = corpora.Dictionary(data_lemmatized)

texts = data_lemmatized

corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]


In [12]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.021*"let" + 0.016*"life" + 0.014*"game" + 0.013*"co" + 0.013*"look" + 0.011*"back" + 0.011*"give" + 0.011*"way" + 0.009*"big" + 0.009*"watch"'), (1, '0.034*"http" + 0.028*"go" + 0.021*"make" + 0.014*"say" + 0.012*"m" + 0.011*"never" + 0.011*"thing" + 0.011*"come" + 0.011*"really" + 0.010*"take"'), (2, '0.036*"get" + 0.015*"know" + 0.014*"fuck" + 0.013*"people" + 0.011*"today" + 0.011*"love" + 0.010*"day" + 0.010*"good" + 0.010*"feel" + 0.009*"want"')]


In [13]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

C:\Users\Chuan Wu\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.204589 -0.051901       1        1  44.170978
1     -0.156936 -0.119794       2        1  33.139937
0     -0.047653  0.171695       3        1  22.689085, topic_info=            Term        Freq       Total Category  logprob  loglift
26           get  910.000000  910.000000  Default  30.0000  30.0000
68          http  641.000000  641.000000  Default  29.0000  29.0000
136           go  527.000000  527.000000  Default  28.0000  28.0000
29          make  406.000000  406.000000  Default  27.0000  27.0000
346          let  272.000000  272.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
169        least   58.558151   60.078147   Topic3  -5.3989   1.4577
4557      expect   56.389875   57.747480   Topic3  -5.4366   1.4595
157        https  100.594174  177.436260   Topic3  -4.8578   0.9158
10016  thefuxedo  111.349666  412.425665   Topic3  -4.7562   0.1739
9989   misslusyd   60.106093  218.090163   Topic3  -5.3728   0.1945

[139 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
729       2  0.991196  actually
73        1  0.995246      also
128       2  0.991206    always
99        3  0.986025       amp
81        3  0.993003      back
...     ...       ...       ...
490       1  0.998228      well
2559      3  0.979427       win
368       1  0.994407      work
10        1  0.039787      year
10        2  0.954899      year

[145 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])